In [3]:
import h5py
import os
import numpy as np

test_case_dir = '/data4/embedding/DORID/Gigapath_tile/AI-DRBR-N065.h5'

def read_assets_from_h5(h5_path: str) -> tuple:
    '''Read the assets from the h5 file, excluding entries with NaN in features'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            data = f[key][:]
            has_nan = np.isnan(data).any()
            if key == 'features' and has_nan:
                print(f"NaN detected in 'features' of file: {h5_path}, skipping entry.")
                continue
            assets[key] = data
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs

assets, attrs = read_assets_from_h5(test_case_dir)

In [4]:
assets

{'coords': array([[14848., 34816.],
        [69632., 40448.],
        [81408., 71680.],
        ...,
        [84480., 49152.],
        [39424., 53248.],
        [39424., 36352.]], dtype=float32),
 'features': array([[-1.1433027 , -0.2955507 , -1.4852103 , ...,  1.2046198 ,
          0.69216824,  1.0772749 ],
        [-1.3890816 , -0.01468386,  0.00973946, ...,  0.31391716,
          1.7117717 , -0.53996754],
        [-1.1238862 ,  1.3188025 , -0.08327063, ..., -0.06665339,
         -0.15629782,  1.0184572 ],
        ...,
        [-1.1291289 , -0.09150105, -0.40298152, ..., -0.18391843,
          0.91654134,  0.2604831 ],
        [-1.6193124 , -1.7648406 ,  0.52067524, ...,  1.209168  ,
          1.2896625 ,  0.43685606],
        [-1.3564286 , -1.7954583 , -0.02657186, ...,  1.3022978 ,
          1.008355  ,  0.56898487]], dtype=float32)}

In [2]:
import os
print(len(os.listdir('/home/yuhaowang/project/FMBC/downstream/finetune/outputs')))

70


In [1]:
import pandas as pd 
data = pd.read_csv('/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/biomarker/TCGA-BRCA_PIK3A.csv')
data['label'].value_counts()

label
True     304
False     27
Name: count, dtype: int64

In [1]:
import pandas as pd
data = pd.read_csv('/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/biomarker/TCGA-GENE-MUTATION.csv')
#select slide_id and other column to several new csv
# for column in data.columns.drop('slide_id'):
#     data[['slide_id', column]].to_csv('./TCGA-BRCA_{column}.csv'.format(column=column), index=False)

In [7]:
data['GATA3'].value_counts()


GATA3
0    878
1     97
Name: count, dtype: int64

In [5]:
task = yaml.load(open('all_task.yaml', 'r'), Loader=yaml.FullLoader)
print(task)

{'AHSL-GRADE-1': {'csv_dir': 'dataset_csv/subtype', 'embedding_dir': '/data4/embedding/AHSL', 'folds': 1, 'task_cfg': 'task_configs/subtype/AHSL-GRADE-1.yaml'}, 'AHSL-GRADE-3': {'csv_dir': 'dataset_csv/subtype', 'embedding_dir': '/data4/embedding/AHSL', 'task_cfg': 'task_configs/subtype/AHSL-GRADE-3.yaml'}, 'AHSL-NON-IDC-GRADE': {'csv_dir': 'dataset_csv/subtype', 'embedding_dir': '/data4/embedding/AHSL', 'task_cfg': 'task_configs/subtype/AHSL-NON-IDC-GRADE.yaml'}, 'AHSL_SUBTYPE': {'csv_dir': 'dataset_csv/subtype', 'embedding_dir': '/data4/embedding/AHSL', 'task_cfg': 'task_configs/subtype/AHSL_SUBTYPE.yaml'}, 'AIDPATH_CERB2': {'csv_dir': 'dataset_csv/biomarker', 'embedding_dir': '/data4/embedding/AIDPATH', 'task_cfg': 'task_configs/biomarker/AIDPATH_CERB2.yaml'}, 'AIDPATH_GRADE': {'csv_dir': 'dataset_csv/subtype', 'embedding_dir': '/data4/embedding/AIDPATH', 'task_cfg': 'task_configs/subtype/AIDPATH_GRADE.yaml'}, 'AIDPATH_IDC': {'csv_dir': 'dataset_csv/subtype', 'embedding_dir': '/data

In [1]:
import os

def delete_pt_files(root_dir):
    """
    递归遍历 root_dir 目录，删除所有以 .pt 结尾的文件。
    """
    deleted_files = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".pt"):
                file_path = os.path.join(dirpath, filename)
                try:
                    os.remove(file_path)
                    deleted_files.append(file_path)
                    print(f"已删除: {file_path}")
                except Exception as e:
                    print(f"删除失败: {file_path}，错误信息: {e}")

    print(f"\n共删除 {len(deleted_files)} 个 .pt 文件。")

# 使用示例：
# 将下面的路径替换为你要处理的目录
target_directory = "/home/yuhaowang/project/FMBC/downstream/finetune/ablation_results"
delete_pt_files(target_directory)


已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/UNI/LR_Same_0.25_MeanPool/0.0001/fold_2/checkpoint.pt
已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/UNI/LR_Same_0.25_MeanPool/0.0001/fold_3/checkpoint.pt
已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/UNI/LR_Same_0.25_MeanPool/0.0001/fold_1/checkpoint.pt
已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/UNI/LR_Same_0.25_MeanPool/0.0001/fold_0/checkpoint.pt
已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/UNI/LR_Same_0.25_MeanPool/0.0001/fold_4/checkpoint.pt
已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/CONCH/LR/0.001/fold_1/checkpoint.pt
已删除: /home/yuhaowang/project/FMBC/downstream/finetune/ablation_results/prop75/AIDPATH_CERB2/CONCH/LR/0.001/fold_0/checkpoint.pt
已删除: /home/yuh